### Update scikit learn package

In [ ]:
!pip install -U scikit-learn

#### Please restart kernel from the Kernel menu at the top of the page before proceeding

In [ ]:
# Lets use the Python image processing Pillow library
from PIL import Image
img = Image.open('puppy-image.jpg')
display(img)

In [ ]:
# Print the size of the image as width, height
img.size

### Resize Images

In [2]:
# Resize the image to a smaller size
img_smaller = img.resize((600, 400))
img_smaller.save('puppy_image_small.jpg')

print(img.size)
print(img_smaller.size)

(1920, 1280)
(600, 400)


In [ ]:
display(img_smaller)

In [ ]:
# Get the pixel values from the image
width, height = img.size
pixels = {'R':[],'G':[],'B':[]}
# Let us now get the RGB value for each pixel position
for w in range(1,width):
    for h in range(1, height):
        pixpos = img.getpixel(((w,h)))
        pixels['R'].append(pixpos[0])
        pixels['G'].append(pixpos[1])
        pixels['B'].append(pixpos[2])

In [ ]:
# Print the first 5 pixels in the first width position and up to 10 pixels height
for i in range(1,6):
    print("R: "+str(pixels['R'][i])+' '+"G: "+str(pixels['G'][i])+' '+"B: "+str(pixels['B'][i])+' ')

In [ ]:
# How many pixels do we have?
len(pixels['R'])

In [ ]:
# Get the pixel values from the smaller image
width, height = img_smaller.size
pixels = {'R':[],'G':[],'B':[]}
# Let us now get the RGB value for each pixel position
for w in range(1,width):
    for h in range(1, height):
        pixpos = img.getpixel(((w,h)))
        pixels['R'].append(pixpos[0])
        pixels['G'].append(pixpos[1])
        pixels['B'].append(pixpos[2])

In [ ]:
# How many pixels do we have?
len(pixels['R'])

### Cropping Images

In [ ]:
# Cropping and tiling our image
# define cropping coordinates
crop_coord = (130,75,460,350)
# crop the image
img_cropped = img_smaller.crop(crop_coord)
display(img_cropped)

In [ ]:
# Get the pixel values from the smaller image
width, height = img_cropped.size
pixels = {'R':[],'G':[],'B':[]}
# Let us now get the RGB value for each pixel position
for w in range(1,width):
    for h in range(1, height):
        pixpos = img.getpixel(((w,h)))
        pixels['R'].append(pixpos[0])
        pixels['G'].append(pixpos[1])
        pixels['B'].append(pixpos[2])

In [ ]:
# How many pixels do we have?
len(pixels['R'])

### Tiling our image

In [ ]:
# make a folder called tiles
!mkdir tiles

In [ ]:
# We will now use the crop function to create multiple tiles of our image
import math
# get the dimensions of the input image - we are using the resize image for this example
w, h = img_smaller.size
# How many images do we want? Feel free to change this value but it should be a perfect squared number
# And the width and height of the input image must be divisible by this square root
tiles = 16
divisor = int(math.sqrt(tiles))

left_pixel = 0
top_pixel = 0
right_pixel = w/divisor
bottom_pixel = h/divisor
# traverse from the left to right of the image or traverse through the columns
for i in range(divisor):
    if right_pixel <= w:
        # tiling position to attach to the image name referring to the row and column number 
        # such as puppy_tiled_r0_c1.jpg, puppy_tiled_r0_c2.jpg... 
        top_pixel = 0
        # initialize to first row here
        j = 0
        bottom_pixel = h/divisor
        #print("Column traversal: " + "left is: " + str(left_pixel) + "top is : " + str(top_pixel) + "right is : " + str(right_pixel) + "bottom is : " + str(bottom_pixel))
        tiled_img = img_smaller.crop((left_pixel, top_pixel, right_pixel, bottom_pixel)) 
        tiled_img.save('tiles/puppy_tiled_' + 'r'+str(j) + '_c'+str(i) +".jpg")
        # now traverse down the height of the image - traverse through the rows for each column
        for j in range(1,divisor):
            top_pixel += h/divisor
            bottom_pixel += h/divisor
            if bottom_pixel <= h:    
                #print("Row traversal:  " + "left is: " + str(left_pixel) + "top is : " + str(top_pixel) + "right is : " + str(right_pixel) + "bottom is : " + str(bottom_pixel))
                tiled_img = img_smaller.crop((left_pixel, top_pixel, right_pixel, bottom_pixel)) 
                tiled_img.save('tiles/puppy_tiled_' + 'r'+str(j) + '_c'+str(i) +".jpg")
        # increment pixel positions
        left_pixel += w/divisor
        right_pixel += w/divisor

#### Display the tiles

In [ ]:
# Lets use matplot lib to display the tiled images 
import os
import matplotlib.pyplot as plt
images = []

for tile in sorted(os.listdir('./tiles')):
    if tile.endswith(".jpg"):
        images.append(plt.imread('./tiles/'+tile))

f, pltarr = plt.subplots(divisor, divisor, figsize=(12,8))

for j, row in enumerate(pltarr):
    for i, axis in enumerate(row):
        axis.imshow(images[j*divisor+i])
        axis.set_title(f'tile {j*divisor+i+1}')

title = 'Puppy tiled images'
f.suptitle(title, fontsize=16)
plt.show() 

### Rotating images

In [ ]:
# rotate images to various angles 
angles = [45,90,180,225]
for angle in angles:
    display(img_smaller.rotate(angle))

### Convert to Grayscale

In [ ]:
# Convert image to gray scale aka black and white
gray_img = img_smaller.convert('L')
display(gray_img)

### Convert to RecordIo

In [ ]:
# first install mxnet. You can also use a kernel that has MXNet already installed - select a kernel from the top right of SageMaker Studio
!pip install mxnet

In [ ]:
# we have to install opencv package for recordIo conversion
!pip install opencv-python-headless

In [ ]:
# Download a copy of the im2rec python file we need
!curl -O https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py

In [ ]:
# create a folder to label our tiled puppy images as labrador class
!mkdir tiles/labrador

In [ ]:
# move the images to the labrador folder
!mv tiles/*.jpg tiles/labrador

In [ ]:
# create a RecordIO list file to indicate the label for the images
!python im2rec.py ./puppy ./tiles/ --recursive --list

In [ ]:
# now generate the recordIO file for our images which will be the input for training
!python im2rec.py ./puppy ./tiles/ --recursive --pass-through --pack-label

In [ ]:
#you should see a puppy.rec file and a puppy.idx file created which are inputs to your training
!ls -lt puppy*

### Principal Component Analysis

In [3]:
# We need a numpy array of our image to get started. Import numpy first
import numpy as np
# Let us convert our grayscale image to a transposed numpy array first
seq = img_smaller.getdata()
img_pix_array = np.array(seq)
img_pix_array.shape

(240000, 3)

In [4]:
# Let us use Scikit learn PCA to see the variability with all 3 features
from sklearn.decomposition import PCA
pca_3 = PCA(n_components=3)
pca_3.fit(img_pix_array)

PCA(n_components=3)

In [5]:
print(pca_3.explained_variance_ratio_ *100)

[94.97632805  4.43190176  0.59177018]


In [ ]:
# Component 1 accounts for 95% variability and is sufficient for us. We will use number of components as 1 and run the transform
from sklearn.decomposition import PCA
pca_1 = PCA(n_components=1)
pca_1.fit(img_pix_array)
img_reduced = pca_1.transform(img_pix_array)
img_reduced.shape

In [ ]:
print(img_reduced[0])

### END OF NOTEBOOK
Please go back to the Chapter 03 in the book for next steps